In [50]:
import os
import json
import pandas as pd
import rouge
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration

rouge_scoring = rouge.Rouge()

In [3]:
from datasets import load_metric

rouge = load_metric("rouge")
predictions = ["I really loved reading the Hunger Games"]
references = ["I loved reading the Hunger Games"]
rouge.compute(predictions=predictions, references=references)

C:\Users\Asus ROG\AppData\Local\Temp\ipykernel_21920\1865291137.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


{'rouge1': AggregateScore(low=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), mid=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), high=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)),
 'rouge2': AggregateScore(low=Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272), mid=Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272), high=Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272)),
 'rougeL': AggregateScore(low=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), mid=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), high=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)),
 'rougeLsum': AggregateScore(low=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), mid=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.92307692307

In [4]:
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")
model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")


In [5]:
import tarfile

In [22]:
directory_path = r"dataset\liputan6_data\canonical\train"

if os.path.exists(directory_path):
    files_in_directory = os.listdir(directory_path)
    print(files_in_directory[:5])
else:
    print("Directory not found:", directory_path)

['100000.json', '100002.json', '100003.json', '100004.json', '100005.json']


In [26]:
# Load sample json data
data_sample = json.load(open(
    r"dataset\liputan6_data\canonical\train\100002.json", "r"
))


In [27]:
# Define sample preprocessed article 
ARTICLE_TO_SUMMARIZE = " ".join(sum(data_sample['clean_article'], []))
print(ARTICLE_TO_SUMMARIZE)

Liputan6 . com , Jakarta : Perdana Menteri Jepang Junichiro Koizumi meminta maaf atas kekejaman tentara Jepang pada masa Perang Dunia II di Asia . Permohonan maaf secara formal itu Koizumi utarakan di depan pemimpin negara-negara Asia dan Afrika dalam Konferensi Tingkat Tinggi Asia-Afrika di Jakarta Convention Center , Jumat ( 22/4 ) . Koizumi mengatakan , pada masa silam Jepang terlalu ambisius untuk berkuasa . Ternyata , ambisi itu justru menimbulkan kerusakan dan penderitaan luar biasa bagi penduduk di sejumlah negara , terutama di Asia . " Untuk itu Jepang minta maaf , " kata Koizumi . Selain meminta maaf , Koizumi berharap KAA dapat membantu mempererat persahabatan antara Jepang dan negara-negara di Asia serta Afrika . Jepang , tambah Koizumi , siap memberikan sumbangan dan bantuan bagi negara-negara Asia dan Afrika yang membutuhkan . Pernyataan Koizumi membuat heran para pengamat . Menurut mereka , ini kejadian langka . Mereka menduga , pernyataan Koizumi terkait dengan makin pan

In [28]:
# Encode sentence using pretrained model
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors = "pt")


In [30]:
summary_ids = model.generate(
    input_ids, min_length = 25, max_length = 75, 
    num_beams = 10, Arepetition_penalty = 2.5, 
    length_penalty = 1.0, early_stopping = True, 
    no_repeat_ngram_size = 3, use_cache = True, 
    temperature = 0.9, top_k = 10, top_p = 0.9
)

In [31]:
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)


Perdana Menteri Jepang Junichiro Koizumi meminta maaf atas kekejaman tentara Jepang pada masa Perang Dunia II di Asia. Pernyataan itu membuat heran para pengamat.


In [51]:
rouge_scoring.get_scores(
    ARTICLE_TO_SUMMARIZE, summary_text
)

[{'rouge-1': {'r': 1.0, 'p': 0.1513157894736842, 'f': 0.2628571405740408},
  'rouge-2': {'r': 0.8695652173913043,
   'p': 0.09389671361502347,
   'f': 0.16949152366453607},
  'rouge-l': {'r': 1.0, 'p': 0.1513157894736842, 'f': 0.2628571405740408}}]